# Refine Documents Chain

## 介绍


`RefineDocumentsChain` 的结构和工作原理不仅限于生成总结。实际上，它可以应用于任何需要逐步处理多个输入的任务，无论是生成总结、回答问题、进行翻译，还是其他需要递归处理并逐步扩展上下文的应用场景。

### **泛化的工作原理**

1. **初步处理**：
   - `initial_llm_chain` 使用 `initial_prompt_template` 对第一个文档（或第一个输入）进行初步处理。这个过程可以是生成一个总结、翻译一段文字、提取关键信息等。

2. **细化处理**：
   - 随后的文档（或输入）与之前生成的结果结合，通过 `refine_llm_chain` 和 `refine_prompt_template` 进行进一步的处理。这个处理会在之前结果的基础上，结合新的内容，生成一个更为复杂或完整的输出。

### **应用场景示例**

- **总结生成**：
  - 初步总结一个段落，然后逐步添加更多段落的总结内容，形成一个完整的文档总结。

- **问题回答**：
  - 初步回答一个简单的问题，然后通过后续的输入不断细化答案，直到生成一个更详细、全面的回答。

- **翻译文本**：
  - 初步翻译一段文字，然后逐步翻译更多的内容，将之前的翻译结果和新的内容结合，生成一个连贯的完整翻译。

- **信息提取**：
  - 初步提取一段文字中的关键信息，然后逐步结合更多内容，提取出更为复杂的关系或信息。

### **递归处理的优势**

- **保持上下文**：每一步都能保持对之前结果的记忆，使得最终输出能够综合多个输入的内容。
- **递归扩展**：通过逐步处理，可以将复杂任务拆分为多个简单步骤，逐步扩展并完善最终结果。
- **灵活性**：可以通过调整模板和链条的定义，应用于广泛的任务，不局限于总结生成。

因此，`RefineDocumentsChain` 是一种非常通用的链式处理机制，能够递归地处理多个输入，并结合之前的结果生成新的输出。这种结构不仅适用于总结生成，而且可以扩展到许多需要递归处理并逐步构建输出的场景中。

## 源码分析

### 类的定义和文档字符串

In [ ]:
class RefineDocumentsChain(BaseCombineDocumentsChain):
    """Combine documents by doing a first pass and then refining on more documents.

    This algorithm first calls `initial_llm_chain` on the first document, passing
    that first document in with the variable name `document_variable_name`, and
    produces a new variable with the variable name `initial_response_name`.

    Then, it loops over every remaining document. This is called the "refine" step.
    It calls `refine_llm_chain`,
    passing in that document with the variable name `document_variable_name`
    as well as the previous response with the variable name `initial_response_name`.
    """

`RefineDocumentsChain` 是一个链条类，它用于逐步处理多个文档。首先，它会对第一个文档进行初步处理，然后对每个后续文档进行细化处理。在初步处理步骤中，链条会将第一个文档传递给 `initial_llm_chain`，生成一个初始响应。然后，在细化步骤中，它会依次将每个剩余文档与之前生成的响应一起传递给 `refine_llm_chain` 进行进一步处理。

### 成员变量

#### 初步处理链条`initial_llm_chain`

In [ ]:
    initial_llm_chain: LLMChain
    """LLM chain to use on initial document."""

`initial_llm_chain` 是一个 LLMChain 对象，用于初步处理第一个文档。这一链条负责生成对第一个文档的**初步总结**

#### 细化处理链条`refine_llm_chain`

In [ ]:
    refine_llm_chain: LLMChain
    """LLM chain to use when refining."""

`refine_llm_chain` 是一个 LLMChain 对象，用于细化处理后续的文档。它**接收前一步生成的总结**，并结合当前文档**生成更详细的总结**。

#### 文档变量名 `document_variable_name`

In [ ]:
    document_variable_name: str
    """The variable name in the initial_llm_chain to put the documents in.
    If only one variable in the initial_llm_chain, this need not be provided."""

`document_variable_name` 定义了文档内容在 LLM 链中使用的变量名。它是链条在处理文档时用来标识文档内容的关键字。

#### 初步响应变量名 `initial_response_name`

In [ ]:
    initial_response_name: str
    """The variable name to format the initial response in when refining."""

`initial_response_name` 是在细化过程中用于传递初步总结结果的变量名。这个变量名用于标识初步处理的结果，以便在细化处理步骤中使用

#### 文档提示模板 `document_prompt`

In [ ]:
    document_prompt: BasePromptTemplate = Field(
        default_factory=_get_default_document_prompt
    )
    """Prompt to use to format each document, gets passed to `format_document`."""

`document_prompt` 是一个提示模板，用于格式化文档的内容。在处理文档时，链条会使用这个模板来生成输入给 LLM 的格式化文本。

#### 是否返回中间步骤 `return_intermediate_steps`

In [ ]:
    return_intermediate_steps: bool = False
    """Return the results of the refine steps in the output."""

`return_intermediate_steps` 是一个布尔值，用于决定是否在最终输出中包含每个细化步骤的中间结果。如果设置为 `True`，链条会返回每个文档处理后的中间结果。

### 输出键设置函数

In [ ]:
    @property
    def output_keys(self) -> List[str]:
        """Expect input key.

        :meta private:
        """
        _output_keys = super().output_keys
        if self.return_intermediate_steps:
            _output_keys = _output_keys + ["intermediate_steps"]
        return _output_keys

`output_keys` 方法返回链条的输出键名列表。如果 `return_intermediate_steps` 为 `True`，链条会在输出中添加 `intermediate_steps`，这是一个包含中间处理结果的列表。

### 初始化变量校验函数

In [ ]:
    @root_validator(pre=True)
    def get_default_document_variable_name(cls, values: Dict) -> Dict:
        """Get default document variable name, if not provided."""
        if "initial_llm_chain" not in values:
            raise ValueError("initial_llm_chain must be provided")

        llm_chain_variables = values["initial_llm_chain"].prompt.input_variables
        if "document_variable_name" not in values:
            if len(llm_chain_variables) == 1:
                values["document_variable_name"] = llm_chain_variables[0]
            else:
                raise ValueError(
                    "document_variable_name must be provided if there are "
                    "multiple llm_chain input_variables"
                )
        else:
            if values["document_variable_name"] not in llm_chain_variables:
                raise ValueError(
                    f"document_variable_name {values['document_variable_name']} was "
                    f"not found in llm_chain input_variables: {llm_chain_variables}"
                )
        return values

这个校验器在类实例化时确保 `document_variable_name` 被正确设置。如果没有提供 `document_variable_name`，但 `initial_llm_chain` 只有一个输入变量，那么这个变量名会自动设为 `initial_llm_chain` 的输入变量名。如果 `document_variable_name` 不存在于 `initial_llm_chain` 的输入变量中，则会抛出错误。

### 文档组合逻辑函数


#### 同步文档组合`combine_docs`

In [ ]:
    def combine_docs(
        self, docs: List[Document], callbacks: Callbacks = None, **kwargs: Any
    ) -> Tuple[str, dict]:
        """Combine by mapping first chain over all, then stuffing into final chain.

        Args:
            docs: List of documents to combine
            callbacks: Callbacks to be passed through
            **kwargs: additional parameters to be passed to LLM calls (like other
                input variables besides the documents)

        Returns:
            The first element returned is the single string output. The second
            element returned is a dictionary of other keys to return.
        """
        inputs = self._construct_initial_inputs(docs, **kwargs)
        res = self.initial_llm_chain.predict(callbacks=callbacks, **inputs)
        refine_steps = [res]
        for doc in docs[1:]:
            base_inputs = self._construct_refine_inputs(doc, res)
            inputs = {**base_inputs, **kwargs}
            res = self.refine_llm_chain.predict(callbacks=callbacks, **inputs)
            refine_steps.append(res)
        return self._construct_result(refine_steps, res)

`combine_docs` 方法通过初步处理第一个文档，然后逐步细化剩余文档来组合多个文档。它首先构建初始输入，并使用 `initial_llm_chain` 生成初步结果。接着，对剩余的每个文档，它构建细化步骤的输入，并使用 `refine_llm_chain` 生成最终结果。如果 `return_intermediate_steps` 为 `True`，它会返回中间步骤的结果。

#### 异步文档组合`acombine_docs`

In [ ]:
    async def acombine_docs(
        self, docs: List[Document], callbacks: Callbacks = None, **kwargs: Any
    ) -> Tuple[str, dict]:
        """Async combine by mapping a first chain over all, then stuffing
         into a final chain.

        Args:
            docs: List of documents to combine
            callbacks: Callbacks to be passed through
            **kwargs: additional parameters to be passed to LLM calls (like other
                input variables besides the documents)

        Returns:
            The first element returned is the single string output. The second
            element returned is a dictionary of other keys to return.
        """
        inputs = self._construct_initial_inputs(docs, **kwargs)
        res = await self.initial_llm_chain.apredict(callbacks=callbacks, **inputs)
        refine_steps = [res]
        for doc in docs[1:]:
            base_inputs = self._construct_refine_inputs(doc, res)
            inputs = {**base_inputs, **kwargs}
            res = await self.refine_llm_chain.apredict(callbacks=callbacks, **inputs)
            refine_steps.append(res)
        return self._construct_result(refine_steps, res)

`acombine_docs` 是 `combine_docs` 方法的异步版本，用于在异步环境中执行相同的逻辑。它与同步方法的工作流程相同，只是使用 `await` 关键字处理异步操作。

### 结果构造函数

In [ ]:
    def _construct_result(self, refine_steps: List[str], res: str) -> Tuple[str, dict]:
        if self.return_intermediate_steps:
            extra_return_dict = {"intermediate_steps": refine_steps}
        else:
            extra_return_dict = {}
        return res, extra_return_dict

`_construct_result` 方法根据是否需要返回中间步骤来构建最终结果。如果 `return_intermediate_steps` 为 `True`，它会在返回的字典中包含一个 `intermediate_steps` 键，存放所有细化步骤的结果。

### 构建细化步骤的输入函数

In [ ]:
    def _construct_refine_inputs(self, doc: Document, res: str) -> Dict[str, Any]:
        return {
            self.document_variable_name: format_document(doc, self.document_prompt),
            self.initial_response_name: res,
        }

`_construct_refine_inputs` 方法构建细化步骤的输入。它将当前文档格式化后，与**之前生成的总结结果一起作为输入**，传递给 `refine_llm_chain` 处理

### 构建初始化步骤的输入函数

In [ ]:
    def _construct_initial_inputs(
        self, docs: List[Document], **kwargs: Any
    ) -> Dict[str, Any]:
        base_info = {"page_content": docs[0].page_content}
        base_info.update(docs[0].metadata)
        document_info = {k: base_info[k] for k in self.document_prompt.input_variables}
        base_inputs: dict = {
            self.document_variable_name: self.document_prompt.format(**document_info)
        }
        inputs = {**base_inputs, **kwargs}
        return inputs

`_construct_initial_inputs` 方法构建初步处理步骤的输入。它将第一个文档的内容和元数据格式化，并构造初步处理链条所需的输入字典。这个字典会包含文档内容及任何其他需要传递的参数。

### 链条类型函数

In [ ]:
    @property
    def _chain_type(self) -> str:
        return "refine_documents_chain"

`_chain_type` 返回链条的类型，这个类型在框架内部可能用于识别链条的类别或选择不同的处理策略。

## demo

In [21]:
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatTongyi
from langchain.docstore.document import Document
import os

In [27]:
# 总结
# 初步处理文档的提示模板
initial_prompt_template = PromptTemplate(
  input_variables=["context"],
  template="总结这个内容:{context}"
)

# 细化处理文档的提示模板
refine_prompt_template = PromptTemplate(
  input_variables=["context", "prev_response"],
  template=(
    "这是前一个内容的总结:{prev_response}"
    "现在，在这个总结的基础上添加下面内容的总结:{context}"
  )
)


In [28]:
# 从环境变量中获取API密钥，用于初始化 ChatTongyi 模型
api_key = os.getenv("KEY_TONGYI")
if not api_key:
    raise ValueError("API Key is not set. Please ensure that the 'KEY_TONGYI' environment variable is set.")


# 初始化 ChatTongyi 模型，设置文本生成的温度参数，温度越低生成的文本越接近输入
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0,  # 设置生成文本的倾向，值越小生成的文本越接近输入
    streaming=True
)
# 初步处理链和细化处理链
initial_chains = LLMChain(llm = llm, prompt=initial_prompt_template)
refine_chains = LLMChain(llm=llm, prompt=refine_prompt_template)

In [29]:
# 创建 RefineDocumentsChain
refine_chain = RefineDocumentsChain(
    initial_llm_chain=initial_chains,
    refine_llm_chain=refine_chains,
    document_variable_name="context",
    initial_response_name="prev_response",
    return_intermediate_steps=True  # 如果需要返回中间步骤的结果，可以设置为 True
)

In [30]:
# 准备一组文档
docs = [
    Document(page_content="LangChain 是一个强大的库，帮助你轻松处理文档。"),
    Document(page_content="OpenAI 的 GPT-3 模型提供了强大的文本生成能力。"),
    Document(page_content="通过组合这些工具，你可以实现强大的 NLP 应用。")
]

In [31]:
# 使用 RefineDocumentsChain 处理文档
final_summary, intermediate_results = refine_chain.combine_docs(docs)

# 打印最终的总结
print("Final Summary:")
print(final_summary)

# 打印中间步骤的结果
if "intermediate_steps" in intermediate_results:
    print("\nIntermediate Steps:")
    for step, result in enumerate(intermediate_results["intermediate_steps"], 1):
        print(f"Step {step}: {result}")


Final Summary:
通过整合GPT-3模型以及其他自然语言处理（NLP）工具和技术，开发者和专业人士能够构建出功能强大、高效且针对性强的NLP应用。以下是基于上述总结基础上的扩展内容：

1. **集成多种NLP技术**：除了GPT-3模型，还可以结合其他NLP技术如命名实体识别、情感分析、语义解析、机器翻译等，以实现更全面的文本理解和处理能力。例如，通过集成情感分析工具，可以增强GPT-3模型在生成评论、反馈或社交媒体分析时的情感洞察力。

2. **个性化定制**：利用GPT-3模型的可微调特性，可以针对特定行业、领域或应用场景进行定制化训练，使模型能够更加精准地满足特定需求。例如，在金融领域，可以针对特定的交易策略、市场分析报告或风险评估报告进行微调，以提供更加专业和个性化的服务。

3. **自动化流程优化**：将GPT-3模型与其他自动化工具结合，可以优化工作流程，提高效率。比如，结合文档生成工具和GPT-3模型，可以自动完成合同起草、报告撰写等工作，减少人工操作的时间和成本。

4. **多模态处理**：虽然GPT-3模型主要处理文本数据，但通过与图像识别、语音合成等技术的集成，可以实现文本、图像、语音等多种模态信息的综合处理，提升应用的交互性和实用性。例如，在智能客服系统中，不仅能够理解文字问题，还能通过语音识别理解口头询问，并通过语音合成提供回复。

5. **跨语言支持**：借助于GPT-3模型的翻译能力以及与其他语言处理工具的集成，可以构建支持多语言的NLP应用，满足全球化市场的需求。这不仅限于翻译服务，还包括多语言文本分析、内容本地化等应用。

6. **持续学习与迭代**：通过持续收集用户反馈和新数据，不断训练和更新模型，可以保持NLP应用的性能和相关性。这要求有良好的数据管理、监控和自动化流程，确保模型能够适应变化的环境和用户需求。

总之，通过整合GPT-3模型与其他NLP工具和技术，可以构建出功能丰富、适应性强的NLP应用，解决从文本生成到多模态处理、从自动化流程优化到多语言支持的广泛问题。

Intermediate Steps:
Step 1: LangChain 是一个功能丰富的库，专门设计用于简化和优化文档的处理流程。它提供了一系列工具和功能，使得用户能够高效地进行文档的读取、解析、存储、检索以及后续的文本分析任务。以